# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
    
# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,52.27,67,0,21.27,CL,Wed Oct 11 14:43:12 2023
1,1,susuman,62.7833,148.1667,3.97,93,6,1.99,RU,Wed Oct 11 14:43:12 2023
2,2,grytviken,-54.2811,-36.5092,29.91,79,65,16.28,GS,Wed Oct 11 14:43:12 2023
3,3,puerto ayora,-0.7393,-90.3518,78.55,87,93,5.99,EC,Wed Oct 11 14:43:13 2023
4,4,keflavik,64.0049,-22.5624,36.03,93,100,10.36,IS,Wed Oct 11 14:43:13 2023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot


map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 650,
    frame_height = 0,
    size = "Humidity",
    scale = 0.6,
    title="Global Humidity",
    color="City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
#Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_df = city_data_df.loc[(city_data_df["Humidity"] <= 60) & (city_data_df["Humidity"] >30) & 
                            (city_data_df["Max Temp"] >18)]

# Drop any rows with null values
ideal_df = ideal_df.dropna(how="any")

# Display sample data
ideal_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,albany,42.6001,-73.9662,60.76,57,92,8.79,US,Wed Oct 11 14:43:14 2023
11,11,taco pozo,-25.6167,-63.2833,77.36,42,99,20.85,AR,Wed Oct 11 14:43:15 2023
20,20,nebraska city,40.6767,-95.8592,75.85,53,40,5.75,US,Wed Oct 11 14:43:18 2023
23,23,hamilton,39.1834,-84.5333,67.80,43,0,6.91,US,Wed Oct 11 14:40:56 2023
29,29,chonchi,-42.6128,-73.8092,52.81,57,61,5.77,CL,Wed Oct 11 14:43:20 2023
35,35,mil'kovo,55.6078,37.7956,42.71,54,100,19.89,RU,Wed Oct 11 14:43:22 2023
49,49,westport,41.1415,-73.3579,69.30,46,0,9.22,US,Wed Oct 11 14:43:27 2023
50,50,tejen,37.3834,60.5054,69.69,36,72,11.43,TM,Wed Oct 11 14:43:27 2023
75,75,san agustin,20.5333,-103.4667,74.52,50,75,14.97,MX,Wed Oct 11 14:43:34 2023
81,81,bauchi,10.5000,10.0000,81.73,42,100,5.08,NG,Wed Oct 11 14:43:36 2023


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head(30)

,City,Country,Lat,Lng,Humidity,Hotel Name
6,albany,US,42.6001,-73.9662,57,
11,taco pozo,AR,-25.6167,-63.2833,42,
20,nebraska city,US,40.6767,-95.8592,53,
23,hamilton,US,39.1834,-84.5333,43,
29,chonchi,CL,-42.6128,-73.8092,57,
35,mil'kovo,RU,55.6078,37.7956,54,
49,westport,US,41.1415,-73.3579,46,
50,tejen,TM,37.3834,60.5054,36,
75,san agustin,MX,20.5333,-103.4667,50,
81,bauchi,NG,10.5000,10.0000,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
albany - nearest hotel: No hotel found
taco pozo - nearest hotel: Hospedaje El Camionero
nebraska city - nearest hotel: Deluxe Inn
hamilton - nearest hotel: North Vista Manor
chonchi - nearest hotel: Hotel Huildín
mil'kovo - nearest hotel: Дрозды
westport - nearest hotel: Norwalk Inn and Conference Center
tejen - nearest hotel: Tejen Myhmanhanasy
san agustin - nearest hotel: Microtel Inn & Suites by Wyndham Guadalajara Sur
bauchi - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
nakonde - nearest hotel: Zwangendaba Executive Lodge
lata - nearest hotel: Residencial Botânico
al jawf - nearest hotel: No hotel found
upata - nearest hotel: Hotel Garibardi
riversdale - nearest hotel: No hotel found
kasama - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
fedorovskiy - nearest hotel: Маяк
fonte boa - nearest hotel: No hotel found
didwana - nearest hotel: No hotel found
kodi

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 650,
    frame_height = 450,
    size = "Humidity",
    scale = 0.7,
    title="Global Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)